# Setup

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# import os
# os.chdir('/content/drive/MyDrive/data')

In [17]:
# # !pip install pyFAI
# # !pip install ipympl

# from google.colab import output
# output.enable_custom_widget_manager()

In [1]:
# importing matplotlib for plots.
%matplotlib widget
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams['figure.constrained_layout.use'] = True

array([[ 0.,  0., -1., ...,  4.,  5.,  2.],
       [ 0.,  0.,  0., ...,  4.,  2.,  4.],
       [ 0., -1.,  0., ...,  5.,  2.,  3.],
       ...,
       [ 0.,  0.,  0., ..., -2.,  0.,  1.],
       [ 0.,  0.,  1., ...,  1.,  0.,  1.],
       [ 0.,  0.,  0., ...,  2.,  1., -1.]], dtype=float32)

In [3]:
#
import os
import glob
import fabio

import xarray as xr
import numpy as np
import pyFAI
import fabio
import scipy
import time


from scipy.ndimage import median_filter

import shutil


def integrator(
        ai,
        tiff_file=None,
        nc_file=None,
        mask=None,
        radial_range = [0.1,10.1],
        delta_q = 0.0025,
        median_filter_size = 2,
        plot = True,
        plot_range = None,
        show_q_limits = True,
        export_ds_as = False,
        export_fig = True,
):


    if tiff_file is not None:
        img = fabio.open(tiff_file).data
        if median_filter_size > 1:
            img = median_filter(img, size=median_filter_size)
        ds = xr.Dataset()
        ds.attrs = {'tiff_source':tiff_file,
                    'median_filter_size':median_filter_size
                    }
        str_ax_title = '%s \n integrated and exported as \n %s'%(tiff_file,export_ds_as)

    if nc_file is not None:
        with xr.open_dataset(nc_file) as ds:
            for k in ['i1d','i2d','radial','azimuthal','Y_obs','Y_calc','Y_bkg_auto','Y_bkg_gsas','Y_bkg','Y_bkg_arpls','X_in_q','X_in_d','X_in_tth']:
                if k in ds.keys():
                    del ds[k]
            da_dark = ds.dexela_img_dark
            dark_from = nc_file
            ds.attrs['dark_from'] = dark_from
            img = (ds.dexela_img.astype('float32') - da_dark.astype('float32')).values
            if median_filter_size > 1:
                img = median_filter(img, size=median_filter_size)
        str_ax_title = tiff_file

    npt = int(np.ceil((radial_range[1] - radial_range[0]) / delta_q ))
    radial_range = [radial_range[0],radial_range[0]+delta_q*npt]

    #integrate
    i2d = ai.integrate2d(
        data=img,
        npt_rad=npt,
        npt_azim=180,
        filename=None,
        correctSolidAngle=True,
        variance=None,
        error_model=None,
        radial_range=radial_range,
        azimuth_range=None,
        mask=mask,
        dummy=np.NaN,
        delta_dummy=None,
        polarization_factor=None,
        dark=None,
        flat=None,
        method='bbox',
        unit='q_A^-1',
        safe=True,
        normalization_factor=1.0,
        metadata=None,
    )

    # ceate new data arrays
    try:
        radial_unit=str(i2d.unit)
        xlabel=str(i2d.unit.label)
    except:
        radial_unit=str(i2d.unit[0])
        xlabel=str(i2d.unit[0].label)

    da_i2d = xr.DataArray(
        data=i2d.intensity.astype('float32'),
        coords=[i2d.azimuthal.astype('float32'),i2d.radial.astype('float32')],
        dims=['azimuthal','radial'],
        attrs={
            'radial_unit':radial_unit,
            'xlabel':xlabel,
            'ylabel':r"Azimuthal angle $\chi$ ($^{o}$)",
            'detector_name'   :ai.__dict__['detector'].name,
            'wavelength_in_nm':ai.__dict__['_wavelength'],
            'detector_dist'   :ai.__dict__['_dist'],
            'detector_poni1'  :ai.__dict__['_poni1'],
            'detector_poni2'  :ai.__dict__['_poni2'],
            'detector_rot1'   :ai.__dict__['_rot1'],
            'detector_rot2'   :ai.__dict__['_rot2'],
            'detector_rot3'   :ai.__dict__['_rot3'],
        }
    )

    da_i1d = xr.DataArray(
        data=da_i2d.mean(dim='azimuthal').astype('float32'),
        coords=[da_i2d.radial],
        dims=['radial'],
        attrs={
            'radial_unit'      :radial_unit,
            'xlabel'           :xlabel,
            'ylabel'           :r"Intensity (a.u.)",
            'wavelength_in_nm' :ai.__dict__['_wavelength'],
        }
    )
    ds['i1d'] = da_i1d.dropna(dim='radial')

    # cropping ds['i2d'] to non NaN range
    ds['i2d'] = da_i2d.sel(radial=slice(ds['i1d'].radial[0],ds['i1d'].radial[-1]))

    if export_ds_as is not None:
            ds.to_netcdf(export_ds_as,engine="h5netcdf",encoding={'i2d': {'zlib': True, 'complevel': 9}}) # pip install h5netcdf

    if plot:

        fig = plt.figure(figsize=(8,6),dpi=128)
        if plot_range is None:
            plot_range = (radial_range[0]-0.1,radial_range[1]+0.1)
        ax1 = fig.add_subplot(2,1,1)
        np.log(ds.i2d).plot.imshow(ax=ax1,
                                    robust=True,
                                    add_colorbar=False,cmap='Greys',
                                    )
        ax1.set_ylabel(ds.i2d.ylabel)
        ax1.set_xlabel(None)
        ax1.set_title(str_ax_title,fontsize=7)
        ax1.set_xlim(plot_range)
        ax1.set_facecolor('#FFF9D3')

        ax2 = fig.add_subplot(2,1,2,sharex=ax1)
        ds.i1d.plot(ax=ax2)
        ax2.set_xlabel(ds.i2d.xlabel)
        ax2.set_ylabel('Intensity (a.u.)')
        ax2.set_yscale('log')
        ax2.set_xlim(plot_range)

        if show_q_limits:
            ax2.axvline(x=ds.i1d.radial[0],color='m',linewidth=0.5,label='q$_{min}$ @ %.3f'%ds.i1d.radial[0])
            ax2.axvline(x=ds.i1d.radial[-1],color='c',linewidth=0.5,label='q$_{max}$ @ %.3f'%ds.i1d.radial[-1])
        ax2.legend()

        if export_fig:
            fig.savefig(export_ds_as.replace('.nc','.png'),dpi=128)

    return ds

# integrations

In [4]:
# we use pyFAI for integrations: https://pyfai.readthedocs.io/en/v2023.1/
# It there exists a poni file and mask, we can load them like this:

ai_file = 'raw/_calibration.poni'
mask_file = 'raw/_mask.edf'

ai = pyFAI.load(ai_file)
mask = fabio.open(mask_file).data

In [8]:
ds = integrator(
    ai,
    tiff_file='raw/NIST-LaB6.tiff',
    mask=mask,
    radial_range=[0.65, 9.65],
    delta_q=0.0025,
    plot=False,
    export_ds_as='integrated/NIST-LaB6.nc'
)


ds = integrator(
    ai,
    tiff_file='raw/NIST-CeO2.tiff',
    mask=mask,
    radial_range=[0.65, 9.65],
    delta_q=0.0025,
    plot=False,
    export_ds_as='integrated/NIST-CeO2.nc'
)



ds = integrator(
    ai,
    tiff_file='raw/NIST-LaB6-CeO2-mix.tiff',
    mask=mask,
    radial_range=[0.65, 9.65],
    delta_q=0.0025,
    plot=False,
    export_ds_as='integrated/NIST-LaB6-CeO2-mix.nc'
)



ds = integrator(
    ai,
    tiff_file='raw/Air_scattering.tiff',
    mask=mask,
    radial_range=[0.65, 9.65],
    delta_q=0.0025,
    plot=False,
    export_ds_as='integrated/Air_scattering.nc'
)



ds = integrator(
    ai,
    tiff_file='raw/Kapton.tiff',
    mask=mask,
    radial_range=[0.65, 9.65],
    delta_q=0.0025,
    plot=False,
    export_ds_as='integrated/Kapton.nc'
)

In [7]:
ls

integrated/  integrator.ipynb  raw/  refiner.ipynb
